# CBRW

In [1]:
import pandas as pd
import numpy as np

In [2]:
with open("cmc-nominal.arff", encoding="utf-8") as f:
    header = []
    for line in f:
        if line.startswith("@attribute"):
            header.append(line.split()[1])
        elif line.startswith("@data"):
            break
    df = pd.read_csv(f, header=None)
    df.columns = header
df

,Wifes_education,Husbands_education,Wifes_religion,Wifes_now_working?,Husbands_occupation,Standard-of-living_index,Media_exposure,Contraceptive_method_used,class_numberofchildren
0,2,3,1,1,2,3,0,1,0
1,1,3,1,1,3,4,0,1,1
2,2,3,1,1,3,4,0,1,0
3,3,2,1,1,3,3,0,1,0
4,3,3,1,1,3,2,0,1,0
...,...,...,...,...,...,...,...,...,...
1468,4,4,1,0,2,4,0,3,0
1469,4,4,1,1,1,4,0,3,0
1470,3,3,1,0,1,4,0,3,0
1471,3,3,1,0,2,2,0,3,0


In [3]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(df.iloc[:,0:8],df.iloc[:,8],test_size=0.3,random_state=0)
X_train = X_train.values
y_train = y_train.values
print(X_train.shape)
print(y_train.shape)

(1031, 8)
(1031,)


The mode-based intra-feature outlierness of a feature value is defined by the frequency of the mode and the extent that the value's frequency deviates from the mode's frequency.

In [4]:
# calculate freq
mode = []
frequence = []
nodes = 0
nodes_freq = []
nodes_value = []
nodes_feature = []
length = X_train.shape[0]
for i in range(0,X_train.shape[1]):
    freq = {}
    for j in range(0, length):
        freq[X_train[j][i]] = freq.get(X_train[j][i],0) + 1
    frequence.append(freq)
    nodes += len(freq)
    nodes_feature.append(nodes)
    key = list(freq.keys())
    value = list(freq.values())
    for j in range(0, len(freq)):
        nodes_freq.append(value[j]/length)
        nodes_value.append(key[j])
print(frequence)
# get mode
for i in range(0,X_train.shape[1]):
    max_freq = 0
    for item in frequence[i]:
        if(frequence[i][item] > max_freq):
            max_freq = frequence[i][item]
    for j in range(0,len(frequence[i])):
        mode.append(max_freq/length)
print(mode)
print(nodes)
print(nodes_freq)
print(nodes_value)
print(nodes_feature)

[{2: 254, 1: 103, 4: 405, 3: 269}, {2: 128, 1: 26, 4: 620, 3: 257}, {1: 872, 0: 159}, {1: 771, 0: 260}, {2: 291, 3: 419, 1: 303, 4: 18}, {2: 158, 1: 97, 3: 310, 4: 466}, {0: 959, 1: 72}, {1: 448, 3: 367, 2: 216}]
[0.39282250242483024, 0.39282250242483024, 0.39282250242483024, 0.39282250242483024, 0.6013579049466538, 0.6013579049466538, 0.6013579049466538, 0.6013579049466538, 0.8457807953443259, 0.8457807953443259, 0.7478176527643065, 0.7478176527643065, 0.4064015518913676, 0.4064015518913676, 0.4064015518913676, 0.4064015518913676, 0.451988360814743, 0.451988360814743, 0.451988360814743, 0.451988360814743, 0.930164888457808, 0.930164888457808, 0.43452958292919497, 0.43452958292919497, 0.43452958292919497]
25
[0.2463627546071775, 0.09990300678952474, 0.39282250242483024, 0.2609117361784675, 0.12415130940834142, 0.02521823472356935, 0.6013579049466538, 0.2492725509214355, 0.8457807953443259, 0.1542192046556741, 0.7478176527643065, 0.2521823472356935, 0.2822502424830262, 0.406401551891367

In [5]:
# calculate delta(v) = base(v) + dev(v)
delta = np.zeros(nodes)
for i in range(0,nodes):
    delta[i] = 1 - mode[i] + (mode[i] - nodes_freq[i]) / mode[i]
print(delta)

[0.980017   1.35285651 0.6071775  0.94297997 1.19219048 1.35670661
 0.3986421  0.98412597 0.1542192  0.97187976 0.25218235 0.91495796
 0.89908771 0.59359845 0.87044809 1.55063902 1.20895585 1.33985713
 0.88277559 0.54801164 0.06983511 0.99475691 0.56547042 0.74627399
 1.08332756]


The homophily outlying couplings are captured by the conditional probability between features.

In [6]:
# conditional probability-based outlierness influence
def joint(value1, value2, feature1, feature2):
    feature1_index = 0
    feature2_index = 0
    for i in range(0, X_train.shape[1]):
        if(feature1 < nodes_feature[i]):
            break
        else:
            feature1_index += 1
    for i in range(0, X_train.shape[1]):
        if(feature2 < nodes_feature[i]):
            break
        else:
            feature2_index += 1
    if(feature1 == feature2):
        return 0
    cnt = 0
    for i in range(0, length):
        if(X_train[i][feature1_index] == value1 and X_train[i][feature2_index] == value2):
            cnt += 1
    return cnt
    
adjency = np.zeros((nodes,nodes))
for i in range(0,nodes):
    for j in range(0,nodes):
        adjency[i][j] = joint(nodes_value[i], nodes_value[j], i, j)/length / nodes_freq[j]
print(adjency)

[[0.         0.         0.         0.         0.515625   0.23076923
  0.12419355 0.40856031 0.2706422  0.11320755 0.25162127 0.23076923
  0.26460481 0.34128878 0.08910891 0.38888889 0.36075949 0.43298969
  0.23548387 0.17596567 0.24504692 0.26388889 0.29910714 0.25340599
  0.125     ]
 [0.         0.         0.         0.         0.3515625  0.61538462
  0.02258065 0.10894942 0.11582569 0.01257862 0.10635538 0.08076923
  0.11683849 0.12649165 0.03630363 0.27777778 0.16455696 0.22680412
  0.09677419 0.05364807 0.06673618 0.54166667 0.16294643 0.07084469
  0.01851852]
 [0.         0.         0.         0.         0.03125    0.03846154
  0.61451613 0.07392996 0.3440367  0.66037736 0.36835279 0.46538462
  0.32646048 0.21957041 0.71287129 0.11111111 0.16455696 0.08247423
  0.35483871 0.56008584 0.4181439  0.05555556 0.29464286 0.38419619
  0.61111111]
 [0.         0.         0.         0.         0.1015625  0.11538462
  0.23870968 0.40856031 0.26949541 0.21383648 0.27367056 0.22307692
  0.29

In [7]:
# transition matrix W
W = np.zeros((nodes,nodes))
for i in range(0,nodes):
    for j in range(0,nodes):
        W[i][j] = delta[j] * adjency[i][j] / delta.dot(adjency[i].T)
print(W)

[[0.         0.         0.         0.         0.1224374  0.06235889
  0.0098609  0.08008319 0.0083132  0.02191404 0.01263853 0.0420546
  0.04738428 0.04035053 0.01544893 0.12010767 0.08686858 0.11555016
  0.04140435 0.01920668 0.00340846 0.05228445 0.03368769 0.03766602
  0.02697145]
 [0.         0.         0.         0.         0.12079565 0.2406222
  0.00259431 0.03090144 0.00514809 0.00352329 0.00772996 0.02129856
  0.03027548 0.02164002 0.00910743 0.12413988 0.05733632 0.08758153
  0.02462141 0.00847319 0.00134319 0.15529311 0.02655571 0.01523732
  0.00578188]
 [0.         0.         0.         0.         0.00750093 0.01050588
  0.04932148 0.01464842 0.01068225 0.12921841 0.01870244 0.08573
  0.05909522 0.02624135 0.12493199 0.03468869 0.04005404 0.02224828
  0.06306683 0.06179652 0.00587921 0.01112664 0.03354482 0.05772594
  0.13329064]
 [0.         0.         0.         0.         0.02938419 0.03798993
  0.0230934  0.09757567 0.01008613 0.05043465 0.01674856 0.04953252
  0.0637327

In [8]:
# outlierness of node v: stationary probability
pi = np.full(25,1/25)
tmp = np.ones(25)
gap = 100
T = 0
alpha = 0.95
while(gap >= 0.001 or T < 100):
    T += 1
    pi_cur = (1-alpha)*1/nodes*tmp + alpha*pi.dot(W)
    gap = (pi_cur - pi).sum()
    gap = np.abs(gap)
    pi = pi_cur
print(pi)
print(gap)
print(T)

[0.04263285 0.09562649 0.01830869 0.03282289 0.0638554  0.08315654
 0.01450747 0.03917125 0.00871672 0.03270632 0.01229329 0.04255331
 0.0376453  0.02624561 0.02401949 0.08075817 0.05392357 0.06927581
 0.0309401  0.02004264 0.00478013 0.07260855 0.02813294 0.02739685
 0.03787962]
0.0
100


# SDRW

Improvements in homophily coupling: Lift is a non-logarithmic pointwise mutual information measure.
1. easier to detect positively influence outlierness pairs: the value > 1
2. larger the correlated rare values than the correlated frequent values

In [9]:
# lift-based outlierness influence
adjency_sdrw = np.zeros((nodes,nodes))
for i in range(0,nodes):
    for j in range(0,nodes):
        adjency_sdrw[i][j] = joint(nodes_value[i], nodes_value[j], i, j)/length / (nodes_freq[j] * nodes_freq[i])
print(adjency_sdrw)

[[0.         0.         0.         0.         2.0929503  0.93670503
  0.50410846 1.65836882 1.09855161 0.45951567 1.02134461 0.93670503
  1.07404551 1.38530998 0.36169798 1.57852143 1.46434267 1.75752902
  0.95584201 0.71425433 0.99465897 1.07113955 1.21409238 1.0285889
  0.50738189]
 [0.         0.         0.         0.         3.51903823 6.15982076
  0.22602568 1.09055192 1.1593814  0.12590829 1.0645864  0.80847647
  1.16951923 1.26614454 0.36338877 2.78047465 1.64716726 2.27024322
  0.96868149 0.53700154 0.66800976 5.42192557 1.63104629 0.70913468
  0.18536498]
 [0.         0.         0.         0.         0.07955247 0.09791073
  1.56436081 0.18820195 0.875807   1.68110878 0.93770796 1.18471985
  0.8310636  0.55895577 1.81474147 0.28285322 0.4189092  0.20995291
  0.90330546 1.42579876 1.06446015 0.14142661 0.75006614 0.97804017
  1.55569273]
 [0.         0.         0.         0.         0.38925999 0.4422362
  0.91490586 1.56589472 1.03289878 0.81957401 1.04890091 0.85498999
  1.1195

Improvements in intra-feaure outlierness: noise-tolerant term

Intuition: the neighbors of outlying values in the value graph are much more likely to be outlying values than noisy values

Method: use subgraph density-based outlier factor instead of delta

Dense subgraph discovery

In [15]:
# Compute the weighted degree of each node
degree = {}
for i in range(0,nodes):
    sum = 0
    for j in range(0,nodes):
        sum += adjency_sdrw[i][j]
    degree[i] = sum
# rank by value
degree = sorted(degree.items(), key=lambda kv:(kv[1],kv[0]), reverse=True)
print(degree)

[(1, 33.771891136238004), (21, 32.93006406008355), (5, 29.408728038794962), (4, 26.778947544678985), (15, 26.541411006448428), (17, 26.495889920920995), (22, 24.60170304900639), (11, 23.874379726276707), (16, 23.794400442881958), (8, 23.665344296108554), (0, 22.81565415445158), (10, 22.705137835496828), (13, 22.658412277649568), (12, 22.32164595251271), (20, 22.254468600285698), (7, 21.47045284571851), (23, 20.081804777308925), (24, 19.863030929503548), (18, 19.761940109675855), (19, 19.73215015814549), (3, 19.59755234467681), (9, 19.351067759706552), (6, 19.259292586972197), (2, 17.544635745725245), (14, 17.108187740486994)]


In [52]:
dense = []
for k in range(1,nodes-1):
    subgraph = np.zeros(k+1)
    for i in range(0,k+1):
        subgraph[i] = degree[i][0]
    subgraph = subgraph.astype(int)
    intra = delta[subgraph.tolist()]
    columns = np.tile(subgraph, k+1)   
    rows = subgraph.repeat(k+1)
    inter = adjency_sdrw[rows,columns].reshape((k+1,k+1))
    dense.append(intra.dot(inter.dot(intra.T)) / (2*k))
print(dense)

[7.296628739758628, 11.531129171487608, 10.906046476322972, 12.750772192423074, 14.369278538718188, 13.069941843123225, 12.426681831447253, 12.666756285929903, 11.455912837015994, 11.595851992273639, 10.779330265484049, 10.525217267318704, 10.540099809642458, 9.841767003587488, 10.054052338174097, 9.951917314710505, 9.946857248447532, 10.001220325925745, 9.788671671740843, 9.96368516075216, 10.069468134778171, 9.84125911910878, 9.705466216403062]


In [53]:
gama = np.zeros(nodes)
cnt = -1
for item in degree:
    sum = 0
    if(nodes-2-cnt == 0):
        break
    for i in range(cnt, nodes-2):
        sum += dense[i]
    if(cnt == -1):
        gama[item[0]] = sum / (nodes-2)
    else:
        gama[item[0]] = sum / (nodes-2-cnt)
    cnt += 1
print(gama)

[10.1860617  11.25145557  9.70546622  9.89496966 10.9642978  10.99006287
  9.77336267  9.92473306 10.27071845  9.87206449 10.0776163  10.53848322
  9.97313312 10.01914013  0.         10.96721037 10.42047081 10.87333869
  9.89496177  9.87371006  9.91643645 10.82947877 10.67911981  9.90856815
  9.90237541]


In [55]:
# transition matrix W_sdrw
W_sdrw = np.zeros((nodes,nodes))
for i in range(0,nodes):
    for j in range(0,nodes):
        W_sdrw[i][j] = gama[j] * adjency_sdrw[i][j] / gama.dot(adjency_sdrw[i].T)
print(W_sdrw)

[[0.         0.         0.         0.         0.09847157 0.04417475
  0.02114166 0.07062705 0.04841639 0.01946612 0.04416734 0.04235962
  0.0459648  0.05955915 0.         0.07428785 0.06547887 0.08200428
  0.04058554 0.03026247 0.04232539 0.04977654 0.05563631 0.04373447
  0.02155984]
 [0.         0.         0.         0.         0.10871318 0.19074169
  0.00622414 0.03049598 0.03355093 0.00350219 0.03022847 0.02400617
  0.03286369 0.03574301 0.         0.08591965 0.0483618  0.06955249
  0.02700676 0.0149394  0.01866448 0.16543922 0.04907713 0.0197978
  0.00517183]
 [0.         0.         0.         0.         0.00550378 0.0067898
  0.0964734  0.01178609 0.05675915 0.1047202  0.05962825 0.07878081
  0.0522989  0.0353374  0.         0.0195742  0.02754444 0.01440494
  0.05639952 0.08883115 0.06660575 0.00966419 0.05054308 0.06114969
  0.09720528]
 [0.         0.         0.         0.         0.02203073 0.02508781
  0.04615606 0.0802213  0.05476043 0.04176423 0.05456329 0.04651012
  0.0576